In [1]:
def findfile(sheetname):
    row = 0
    col = 0
    for name, df_l in df_dicts:
        try:
            col = df_l[sheetname].shape[1]
            df_l[sheetname]
            print(name)
        except:
            continue
        try:
            row += df_l[sheetname].shape[0] 
            if (abs(col - col)<1):
                   continue
            else:
                col += abs(df_l[sheetname].shape[1] - col)
        except:
            continue
    print('rows = %d, col = %d' %(row, col))

In [2]:
import pandas as pd
import numpy as np
import warnings
import re
from datetime import datetime
from collections import OrderedDict
warnings.filterwarnings('default')

In [3]:
# подгрузка всех файлов
import os
dr = 'DATA/source/exp'
file_names = os.listdir(dr)
file_names.remove('prepared')
# файлы в список
df_dicts = {}
for name in file_names:
    df_dicts[name]= pd.read_excel(dr+'\\'+name, None)
#     print(name)

In [4]:
# file_names

In [5]:
# список всех таблиц из всех файлов
sheetname_list = []
for name, dict_ in zip(df_dicts, df_dicts.values()):    
    tmp = dict_.keys()
    tmp = list(tmp)
    tmp = tmp[2:-3]
    sheetname_list.append(tmp)
len(sheetname_list)

30

In [6]:
# dict_sheet_count - словарь всех вкладок(листов) из таблиц без дубликатов и их частоты
# uniq_sheetname - множество всех вкладок(листов) из таблиц без дубликатов (не используется)

dict_sheet_count = {}
uniq_sheetname = set()
for st in sheetname_list:
    for ls in st:
        uniq_sheetname.add(ls)


uniq_sheetname = list(uniq_sheetname)
uniq_sheetname.sort()
print(len(uniq_sheetname))

# добавление частот в словарь
for st in sheetname_list:
    for ls in st:
        counts = dict_sheet_count.get(ls,0)
        dict_sheet_count[ls] = counts + 1
        
        
# df - для вывода словаря в excel            
df = pd.DataFrame.from_dict(data = dict_sheet_count, orient='index')
# df.to_excel('dict_sheet_count.xlsx')

114


In [7]:
ls_mb_convert_ys = [
    'Предел текучести (1141)',
    'Предел текучести (1027)',
    'Предел текучести, Н/мм2 или МПа',
    'Предел текучести , кгс/мм2',
    'Предел текучести, кгс/мм2',
]

ls_mb_convert_tr = [
    'Врем. сопротивление,  Н/мм2 или МПа',
    'Врем. сопротивление (1140)',
    'Врем. сопротивление (1042)',
    'Врем. сопротивление , кгс/мм2',
    'Врем. сопротивление, кгс/мм2'
]

ls_mb_convert = ls_mb_convert_ys+ls_mb_convert_tr

In [8]:
def change_col_name(df):
    for col in ls_mb_convert_tr:
        if col in df:
            df['Врем. сопротивление'] = df[col]
            del df[col]
    for col in ls_mb_convert_ys:
        if col in df:
            df['Предел текучести'] = df[col]
            del df[col]
    return df

In [9]:
ls_sheets = [
    u'Растяжение, кгс мм2.',
    u'Растяжение, н мм2.',
]

In [10]:
scretching = pd.DataFrame()
for filename in file_names:
    tmp = df_dicts[filename][u'Растяжение, кгс мм2.']
    tmp = change_col_name(tmp)
    scretching= pd.concat([tmp, scretching])

In [11]:
scretching2  = pd.DataFrame()
for filename in file_names:
    tmp = df_dicts[filename][u'Растяжение, н мм2.']
    tmp = change_col_name(tmp)
    scretching2= pd.concat([tmp, scretching2])

In [12]:
scr_df = scretching[[
    '№ плавки',
    '№ партии',
    'Дата форм. цехом',
    'Марка стали',
    'Тип предела текучести (1186)',
    'Врем. сопротивление',
    'Предел текучести',
    'ГОСТ, ТУ', 'Гр. прочн.']].copy()

In [13]:
scr_df2 = scretching2[[
    '№ плавки',
    '№ партии',
    'Дата форм. цехом',
    'Марка стали',
    'Тип предела текучести (1186)',
    'Врем. сопротивление',
    'Предел текучести',
    'ГОСТ, ТУ', 'Гр. прочн.']].copy()

In [14]:
scr_df[scr_df['№ партии']==712261]

,№ плавки,№ партии,Дата форм. цехом,Марка стали,Тип предела текучести (1186),Врем. сопротивление,Предел текучести,"ГОСТ, ТУ",Гр. прочн.


In [16]:
# scr_df= scr_df[scr_df['Тип предела текучести (1186)']==0.2]

In [17]:
# scr_df2= scr_df2[scr_df2['Тип предела текучести (1186)']==0.2]
# .describe()

In [18]:
# scr_df2.describe()

In [19]:
# scr_df.describe()

In [15]:
scr_df.loc[:, 'Врем. сопротивление'] = scr_df['Врем. сопротивление'].apply(lambda x: x/9.8 if x>200 else x)
scr_df.loc[:, 'Предел текучести'] = scr_df['Предел текучести'].apply(lambda x: x/9.8 if x>200 else x);

In [16]:
scr_df2['Врем. сопротивление'] = scr_df2['Врем. сопротивление'].apply(lambda x: x/9.8 if x>200 else x)
scr_df2['Предел текучести'] = scr_df2['Предел текучести'].apply(lambda x: x/9.8 if x>200 else x)

In [17]:
scr_df.describe()

,Тип предела текучести (1186),Врем. сопротивление,Предел текучести
count,21613.000000,21687.000000,21671.000000
mean,0.200629,70.435213,58.048872
std,0.014190,15.501042,16.800998
min,0.200000,35.500000,19.000000
25%,0.200000,56.500000,43.500000
50%,0.200000,70.000000,52.000000
75%,0.200000,82.500000,71.000000
max,0.600000,130.000000,122.500000


In [18]:
scr_df = pd.concat([scr_df, scr_df2])

In [19]:
scr_df[scr_df['№ партии']=='712261']

,№ плавки,№ партии,Дата форм. цехом,Марка стали,Тип предела текучести (1186),Врем. сопротивление,Предел текучести,"ГОСТ, ТУ",Гр. прочн.


In [20]:
scr_df['№ партии']=scr_df['№ партии'].apply(lambda x: str(x))

In [21]:
scr_df.shape

(59143, 9)

In [22]:
scr_df.drop_duplicates(inplace = True)
scr_df.shape

(58086, 9)

In [23]:
scr_df['Дата форм. цехом'] = scr_df['Дата форм. цехом'].apply(lambda x: datetime.strptime(x, "%d.%m.%Y"))
# dt = datetime.strptime("01.02.2016", "%d.%m.%Y")
# dt

In [24]:
scr_df.reset_index(inplace=True,drop=True)

In [25]:
def part_in_df(df):
    tmp = df[df['№ партии']=='712261'].copy()
    return tmp

In [26]:
undupl = scr_df[~scr_df[['№ плавки', '№ партии']].duplicated(keep = False)]

In [27]:
dupl_need_to_mean = scr_df[scr_df[['№ плавки', '№ партии']].duplicated(keep = False)]
# sort_values(['№ партии', 'Дата форм. цехом'])

In [28]:
dupl_need_to_mean.shape, undupl.shape, dupl_need_to_mean.shape[0]+undupl.shape[0]

((46112, 9), (11974, 9), 58086)

In [29]:
def mean_func(x):    
    if min(x['Дата форм. цехом'])!=max(x['Дата форм. цехом']):
        x = x[x['Дата форм. цехом']==max(x['Дата форм. цехом'])]
        if abs(min(x['Врем. сопротивление'])-max(x['Врем. сопротивление']))<=5:
            x['Врем. сопротивление'] = x['Врем. сопротивление'].mean()
        else:
            x['Врем. сопротивление'] = None
        if abs(min(x['Предел текучести'])-max(x['Предел текучести']))<=10:
            x['Предел текучести'] = x['Предел текучести'].mean()
        else:
            x['Предел текучести'] = None
    else:
        x['Врем. сопротивление'] = x['Врем. сопротивление'].mean()
        x['Предел текучести'] = x['Предел текучести'].mean()
#     print(x)
    return x.mean()

In [30]:
def mean_func(x):
    if min(x['Дата форм. цехом'])!=max(x['Дата форм. цехом']):
        x = x[x['Дата форм. цехом']==max(x['Дата форм. цехом'])]
        x['Врем. сопротивление'] = x['Врем. сопротивление'].mean()
        x['Предел текучести'] = x['Предел текучести'].mean()
    else:
        x['Врем. сопротивление'] = x['Врем. сопротивление'].mean()
        x['Предел текучести'] = x['Предел текучести'].mean()
    return x.mean()

In [31]:
def isnull(df):
    return df[pd.isnull(df['Тип предела текучести (1186)'])].shape

In [32]:
# dupl_need_to_mean.shape
# part_in_df(dupl_need_to_mean)
# isnull(y)

In [34]:
y = dupl_need_to_mean.groupby([
    '№ плавки', 
    '№ партии', 
    'Тип предела текучести (1186)',
    'ГОСТ, ТУ',
    'Гр. прочн.'
])[
    'Дата форм. цехом',
    'Врем. сопротивление',
    'Предел текучести'].apply(mean_func)

In [36]:
y.reset_index(inplace=True)

In [37]:
y['№ партии'].value_counts().index

Index(['602160', '403218', '202270', '606406', '703203', '701396', '704285',
       '306298', '610146', '704087',
       ...
       '405508', '310025', '405340', '508354', '401111', '209368', '311019',
       '310354', '406308', '304047'],
      dtype='object', length=14966)

In [38]:
y = y[~y[['№ плавки', '№ партии', 'Тип предела текучести (1186)']].duplicated(keep = False)]

In [39]:
y[y[['№ плавки', '№ партии', 'Тип предела текучести (1186)']].duplicated(keep = False)]

,№ плавки,№ партии,Тип предела текучести (1186),"ГОСТ, ТУ",Гр. прочн.,Врем. сопротивление,Предел текучести


In [99]:
# y.reset_index(inplace=True)
# y = y.dropna()
# y

In [100]:
# dupl_need_to_mean.sort_values(['№ партии', 'Дата форм. цехом'])

In [40]:
y.shape

(16298, 7)

In [41]:
tmp = undupl[['№ плавки', '№ партии', 'Тип предела текучести (1186)',
    'Врем. сопротивление',
    'Предел текучести', 'Гр. прочн.']]
y = pd.concat([y, tmp])

In [42]:
y.drop_duplicates(inplace = True)
y.shape

(28272, 7)

In [43]:
y.to_excel('DATA/prepared/table_stretching_05_jule_gruppa_prochnost.xlsx')

In [107]:
y[y['№ партии']=='712261']

,Врем. сопротивление,"ГОСТ, ТУ",Предел текучести,Тип предела текучести (1186),№ партии,№ плавки
